In [1]:
# %load /home/m/m300793/python/my_import.py
# BASIC
import numpy as np
import xarray as xr
import pandas as pd

#xr.set_options(display_style="text") # make it look better when posted online. Comment to get nicer representation of xarrays

# matplotlib
import matplotlib
#%matplotlib inline
import matplotlib as mpl
import matplotlib.pylab as plt
import matplotlib.tri as mtri
import matplotlib.dates as md
import matplotlib.patches as patches
import matplotlib.dates as mdates
import matplotlib.cbook as cbook

from matplotlib import ticker
matplotlib.rcParams['figure.dpi'] = 100
matplotlib.rcParams['axes.linewidth'] = 1.5

## color
from matplotlib import cm
import cmocean
import seaborn as sns

## cartopy
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point


# scipy
from scipy import stats
from scipy import optimize
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
from scipy.stats import binned_statistic
import scipy.spatial.qhull as qhull


# etc
import subprocess
import time
import datetime
from datetime import timedelta



# Dask
import dask # Distributed data libary
from dask import delayed
from dask import compute
from dask_jobqueue import SLURMCluster # Setting up distributed memories via slurm
from dask.utils import format_bytes
from distributed import Client, progress # Libaray to orchestrate distributed resources
from dask.distributed import Client
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})

from pathlib import Path
from getpass import getuser # Libaray to copy things
from pathlib import Path # Object oriented libary to deal with paths
from tempfile import NamedTemporaryFile, TemporaryDirectory # Creating temporary Files/Dirs
import warnings
warnings.filterwarnings(action='ignore')


/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
# Set some user specific variables
account_name = 'mh0731' # Account that is going to be 'charged' fore the computation
queue = 'compute' # Name of the partition we want to use
#queue = 'gpu' # Name of the partition we want to use
#queue = 'shared' # Name of the partition we want to use
job_name = 'PostProc' # Job name that is submitted via sbatch
memory = "500GiB" # Max memory per node that is going to be used - this depends on the partition
cores = 16 # Max number of cores per task that are reserved - also partition dependend
walltime = '4:00:00' # Walltime - also partition dependent

scratch_dir = Path('/scratch') / getuser()[0] / getuser() # Define the users scratch dir
# Create a temp directory where the output of distributed cluster will be written to, after this notebook
# is closed the temp directory will be closed
dask_tmp_dir = TemporaryDirectory(dir=scratch_dir, prefix=job_name)
cluster = SLURMCluster(memory=memory,
                       cores=cores,
                       project=account_name,
                       walltime=walltime,
                       queue=queue,
                       name=job_name,
                       scheduler_options={'dashboard_address': ':12435'},
                       local_directory=dask_tmp_dir.name,
                       job_extra=[f'-J {job_name}', 
                                  f'-D {dask_tmp_dir.name}',
                                  f'--begin=now',
                                  f'--output={dask_tmp_dir.name}/LOG_cluster.%j.o',
                                  f'--output={dask_tmp_dir.name}/LOG_cluster.%j.o'
                                 ],
                       interface='ib0')

cluster.scale(jobs=1)
cluster

dask_client = Client(cluster)
dask_client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /user/m300793/levante-spawner-advanced//proxy/12435/status,
Dashboard: /user/m300793/levante-spawner-advanced//proxy/12435/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://136.172.120.75:38895,Workers: 0
Dashboard: /user/m300793/levante-spawner-advanced//proxy/12435/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
dpath         = '/work/mh0731/m300793/data/dpp0066/'
dataICON_grid = xr.open_mfdataset(dpath+'/icon_grid_0015_R02B09_G.nc')

In [3]:
dpath_out = '/work/mh0731/m300793/data/ngc2009/linked_data/'
dpath     = '/work/mh0731/m300793/data/ngc2009/linked_data/'
# dataICON  = xr.open_mfdataset(dpath+'/*_atm_2d_30min_mean_*.nc')
dataICON  = xr.open_mfdataset(dpath+'/*_atm_2d_30min_inst_20*.nc')
dataICON

<xarray.Dataset>
Dimensions:   (height: 1, height_2: 1, time: 36672, ncells: 20971520)
Coordinates:
  * height    (height) float64 2.0
  * height_2  (height_2) float64 10.0
  * time      (time) datetime64[ns] 2020-01-20 ... 2022-03-01
Dimensions without coordinates: ncells
Data variables:
    psl       (time, ncells) float32 dask.array<chunksize=(48, 20971520), meta=np.ndarray>
    ps        (time, ncells) float32 dask.array<chunksize=(48, 20971520), meta=np.ndarray>
    sit       (time, ncells) float32 dask.array<chunksize=(48, 20971520), meta=np.ndarray>
    sic       (time, ncells) float32 dask.array<chunksize=(48, 20971520), meta=np.ndarray>
    tas       (time, height, ncells) float32 dask.array<chunksize=(48, 1, 20971520), meta=np.ndarray>
    ts        (time, ncells) float32 dask.array<chunksize=(48, 20971520), meta=np.ndarray>
    uas       (time, height_2, ncells) float32 dask.array<chunksize=(48, 1, 20971520), meta=np.ndarray>
    vas       (time, height_2, ncells) float32 dask.array<chunksize=(48, 1, 20971520), meta=np.ndarray>
    cfh_lnd   (time, ncells) float32 dask.array<chunksize=(48, 20971520), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    number_of_grid_used:  15
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    title:                ICON simulation
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    history:              ./icon at 20220512 152214
    references:           see MPIM/DWD publications
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...

In [ ]:
%%time
for YY in np.arange(2020, 2021):
#    for MM in np.arange(1, 8):
    for MM in np.arange(1, 13):
#         rsdt_daily  = dataICON['rsdt'] .sel(time=str(YY)+'-'+str(MM)).resample(time='1D').mean('time')
#         rsut_daily  = dataICON['rsut'] .sel(time=str(YY)+'-'+str(MM)).resample(time='1D').mean('time')
#         rlut_daily  = dataICON['rlut'] .sel(time=str(YY)+'-'+str(MM)).resample(time='1D').mean('time')
        tas_daily  = dataICON['tas'].sel(time=str(YY)+'-'+str(MM)).resample(time='1D').mean('time').isel(height=0)
        ts_daily   = dataICON['ts'] .sel(time=str(YY)+'-'+str(MM)).resample(time='1D').mean('time')
    
        coords = ts_daily
        make_nc = xr.Dataset(
        {
#             'rsdt' : (("time", "ncells"), rsdt_daily.values ,),
#             'rsut' : (("time", "ncells"), rsut_daily.values ,),
#             'rlut' : (("time", "ncells"), rlut_daily.values ,),
            'ts'  : (("time", "ncells"), ts_daily.values ,),
            'tas' : (("time", "ncells"), tas_daily.values ,),
        },
        coords={
                "time": coords['time'].values, "ncells": coords['ncells'].values,
            },
        )
    
        make_nc = make_nc.assign_coords(lon=("ncells", dataICON_grid['clon'].values*180/np.pi))
        make_nc = make_nc.assign_coords(lat=("ncells", dataICON_grid['clat'].values*180/np.pi))

        make_nc['lon'].attrs['standard_name'] = 'longitude'
        make_nc['lon'].attrs['long_name'] = 'center longitude'
        make_nc['lon'].attrs['units'] = 'degree'

        make_nc['lat'].attrs['standard_name'] = 'latitude'
        make_nc['lat'].attrs['long_name'] = 'center latitude'
        make_nc['lat'].attrs['units'] = 'degree'
    
        if MM <= 9:
            MM_str = '0'+str(MM)
        else:
            MM_str = str(MM)
    
        make_nc.to_netcdf(dpath_out +'/radiation_daily_'+str(YY)+'-'+str(MM_str)+'.nc', mode='w')
#         make_nc.to_netcdf(dpath_out +'/temperature_daily_'+str(YY)+'-'+str(MM)+'.nc', mode='w')
        print('Average is done for '+str(YY)+'-'+str(MM_str))

Average is done for 2020-01
